In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days, date_add_times
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature, get_factorize_feature
from func.parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel
from itertools import combinations

In [ ]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'ProductCD']

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths  = glob('../feature/eda_base/*_test.gz')

train_paths = [path for path in train_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('_DT-')
#                or path.count('ProductCD_t')
#                or path.count('D')
#                or path.count('C')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('domain')
               or path.count('uid')
               )
#                and not path.count('fill')
#                and not path.count('bin')
#                and not path.count('129')
              ]
test_paths = [path for path in test_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('_DT-')
#                or path.count('ProductCD_t')
#                or path.count('D')
#                or path.count('C')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('domain')
               or path.count('uid')
               )
#                and not path.count('fill')
#                and not path.count('bin')
#                and not path.count('129')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)
train_length = df_train.shape[0]
del df_train, df_test
gc.collect()

In [3]:
# startdate = datetime.datetime(2017,12,1)
# data['datetime'] = data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
# data['year'] = data['datetime'].map(lambda x: x.year)
# data['month'] = data['datetime'].map(lambda x: x.month)
# data['month'] = data['month'].map(lambda x: 5 if x==6 else x)
# data['week_no'] = data['datetime'].map(lambda x: x.isocalendar()[1])

# data['DT-M'] = data[['year', 'month']].apply(lambda x: str(x[0]) + '-' + str(x[1]), axis=1)
# data['DT-W'] = data[['year', 'week_no']].apply(lambda x: str(x[0]) + '-' + str(x[1]), axis=1)

# prefix = ''
# dir_save = 'eda_base'
# cols_save = ['DT-M', 'DT-W']
# train = data.iloc[:train_length][cols_save]
# test = data.iloc[train_length:][cols_save]
# save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=False, list_ignore=COLUMNS_IGNORE)
# save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=False, list_ignore=COLUMNS_IGNORE)

In [9]:
#========================================================================
# DT-MでCount Regularize
#========================================================================
prefix = '609'
dir_save = 'create'

col_dtm = '__DT-M'
list_dtm = sorted(data[col_dtm].unique())
n_jobs = len(list_dtm)

cols_uid = [col for col in data.columns if col.count('uid') and not col.endswith('cnt')]
cols_feature = cols_uid

# 全体におけるカウントを先に作っておく
for col in cols_feature:
    cnt_map = data[col].value_counts().to_dict()
    data[f"{col}_cnt"] = data[col].map(cnt_map)
    
    
# Parallel微妙だった
# def parallel_dtm(df, col, dtm):
for col in tqdm(cols_feature):
    df = data[[col_dtm, col, f"{col}_cnt"]]
    for dtm in list_dtm:
        tmp = df[df[col_dtm]==dtm]
    
        # local count
        local_cnt_map = tmp[col].value_counts().to_dict()
        df[f"DT-M_{dtm}_{col}_cnt"] = df[col].map(local_cnt_map)
        df[f"DT-M_{dtm}_{col}_cnt"].fillna(0, inplace=True)
        df[f"DT-M_{dtm}_{col}_cnt_ratio_global"] = df[f"DT-M_{dtm}_{col}_cnt"] / df[f"{col}_cnt"]
    
        cols_save = [col for col in df.columns if col.count('_cnt') and col.startswith('DT-M')]
        train = df.iloc[:train_length][cols_save]
        test = df.iloc[train_length:][cols_save]
        save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)


  0%|          | 0/57 [00:00<?, ?it/s]

(590540,) | DT-M_2017-12_130__cnt_uid_cnt
(590540,) | DT-M_2017-12_130__cnt_uid_cnt_ratio_global
(506691,) | DT-M_2017-12_130__cnt_uid_cnt
(506691,) | DT-M_2017-12_130__cnt_uid_cnt_ratio_global
(590540,) | DT-M_2018-1_130__cnt_uid_cnt
(590540,) | DT-M_2018-1_130__cnt_uid_cnt_ratio_global
(506691,) | DT-M_2018-1_130__cnt_uid_cnt
(506691,) | DT-M_2018-1_130__cnt_uid_cnt_ratio_global
(590540,) | DT-M_2018-10_130__cnt_uid_cnt
(590540,) | DT-M_2018-10_130__cnt_uid_cnt_ratio_global
(506691,) | DT-M_2018-10_130__cnt_uid_cnt
(506691,) | DT-M_2018-10_130__cnt_uid_cnt_ratio_global
(590540,) | DT-M_2018-11_130__cnt_uid_cnt
(590540,) | DT-M_2018-11_130__cnt_uid_cnt_ratio_global
(506691,) | DT-M_2018-11_130__cnt_uid_cnt
(506691,) | DT-M_2018-11_130__cnt_uid_cnt_ratio_global
(590540,) | DT-M_2018-12_130__cnt_uid_cnt
(590540,) | DT-M_2018-12_130__cnt_uid_cnt_ratio_global
(506691,) | DT-M_2018-12_130__cnt_uid_cnt
(506691,) | DT-M_2018-12_130__cnt_uid_cnt_ratio_global
(590540,) | DT-M_2018-2_130__cnt_u


  2%|▏         | 1/57 [01:50<1:42:40, 110.01s/it]

(590540,) | DT-M_2017-12_130__cnt_uid2_cnt
(590540,) | DT-M_2017-12_130__cnt_uid2_cnt_ratio_global
(506691,) | DT-M_2017-12_130__cnt_uid2_cnt
(506691,) | DT-M_2017-12_130__cnt_uid2_cnt_ratio_global
(590540,) | DT-M_2018-1_130__cnt_uid2_cnt
(590540,) | DT-M_2018-1_130__cnt_uid2_cnt_ratio_global
(506691,) | DT-M_2018-1_130__cnt_uid2_cnt
(506691,) | DT-M_2018-1_130__cnt_uid2_cnt_ratio_global
(590540,) | DT-M_2018-10_130__cnt_uid2_cnt
(590540,) | DT-M_2018-10_130__cnt_uid2_cnt_ratio_global
(506691,) | DT-M_2018-10_130__cnt_uid2_cnt
(506691,) | DT-M_2018-10_130__cnt_uid2_cnt_ratio_global
(590540,) | DT-M_2018-11_130__cnt_uid2_cnt
(590540,) | DT-M_2018-11_130__cnt_uid2_cnt_ratio_global
(506691,) | DT-M_2018-11_130__cnt_uid2_cnt
(506691,) | DT-M_2018-11_130__cnt_uid2_cnt_ratio_global
(590540,) | DT-M_2018-12_130__cnt_uid2_cnt
(590540,) | DT-M_2018-12_130__cnt_uid2_cnt_ratio_global
(506691,) | DT-M_2018-12_130__cnt_uid2_cnt
(506691,) | DT-M_2018-12_130__cnt_uid2_cnt_ratio_global
(590540,) | DT


  4%|▎         | 2/57 [03:10<1:32:36, 101.03s/it]

...


 96%|█████████▋| 55/57 [1:13:43<02:12, 66.24s/it]

(590540,) | DT-M_2017-12_130__uid3_C6_V187_cnt
(590540,) | DT-M_2017-12_130__uid3_C6_V187_cnt_ratio_global
(506691,) | DT-M_2017-12_130__uid3_C6_V187_cnt
(506691,) | DT-M_2017-12_130__uid3_C6_V187_cnt_ratio_global
(590540,) | DT-M_2018-1_130__uid3_C6_V187_cnt
(590540,) | DT-M_2018-1_130__uid3_C6_V187_cnt_ratio_global
(506691,) | DT-M_2018-1_130__uid3_C6_V187_cnt
(506691,) | DT-M_2018-1_130__uid3_C6_V187_cnt_ratio_global
(590540,) | DT-M_2018-10_130__uid3_C6_V187_cnt
(590540,) | DT-M_2018-10_130__uid3_C6_V187_cnt_ratio_global
(506691,) | DT-M_2018-10_130__uid3_C6_V187_cnt
(506691,) | DT-M_2018-10_130__uid3_C6_V187_cnt_ratio_global
(590540,) | DT-M_2018-11_130__uid3_C6_V187_cnt
(590540,) | DT-M_2018-11_130__uid3_C6_V187_cnt_ratio_global
(506691,) | DT-M_2018-11_130__uid3_C6_V187_cnt
(506691,) | DT-M_2018-11_130__uid3_C6_V187_cnt_ratio_global
(590540,) | DT-M_2018-12_130__uid3_C6_V187_cnt
(590540,) | DT-M_2018-12_130__uid3_C6_V187_cnt_ratio_global
(506691,) | DT-M_2018-12_130__uid3_C6_V18


 98%|█████████▊| 56/57 [1:14:48<01:05, 65.82s/it]

(590540,) | DT-M_2017-12_130__uid3_C6_V258_cnt
(590540,) | DT-M_2017-12_130__uid3_C6_V258_cnt_ratio_global
(506691,) | DT-M_2017-12_130__uid3_C6_V258_cnt
(506691,) | DT-M_2017-12_130__uid3_C6_V258_cnt_ratio_global
(590540,) | DT-M_2018-1_130__uid3_C6_V258_cnt
(590540,) | DT-M_2018-1_130__uid3_C6_V258_cnt_ratio_global
(506691,) | DT-M_2018-1_130__uid3_C6_V258_cnt
(506691,) | DT-M_2018-1_130__uid3_C6_V258_cnt_ratio_global
(590540,) | DT-M_2018-10_130__uid3_C6_V258_cnt
(590540,) | DT-M_2018-10_130__uid3_C6_V258_cnt_ratio_global
(506691,) | DT-M_2018-10_130__uid3_C6_V258_cnt
(506691,) | DT-M_2018-10_130__uid3_C6_V258_cnt_ratio_global
(590540,) | DT-M_2018-11_130__uid3_C6_V258_cnt
(590540,) | DT-M_2018-11_130__uid3_C6_V258_cnt_ratio_global
(506691,) | DT-M_2018-11_130__uid3_C6_V258_cnt
(506691,) | DT-M_2018-11_130__uid3_C6_V258_cnt_ratio_global
(590540,) | DT-M_2018-12_130__uid3_C6_V258_cnt
(590540,) | DT-M_2018-12_130__uid3_C6_V258_cnt_ratio_global
(506691,) | DT-M_2018-12_130__uid3_C6_V25


100%|██████████| 57/57 [1:15:51<00:00, 65.05s/it]


In [30]:
prefix = '609'
dir_save = 'valid_use'
cols_save = [col for col in data.columns if col.count('_cnt') and col.count('DT-M')]
train = data.iloc[:train_length][cols_save]
test = data.iloc[train_length:][cols_save]
save_feature(train, prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
save_feature(test,  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

data.drop(cols_save, axis=1, inplace=True)
del train, test
gc.collect()

2084

In [6]:
#========================================================================
# DT-WでCount Regularize
#========================================================================
prefix = '609'
dir_save = 'valid_use'

col_dtw = 'DT-W'
list_dtw = sorted(data['DT-W'].unique())
n_jobs = len(list_dtw)

cols_C = [col for col in data.columns if col.startswith('C') and not col.count('cnt')]
cols_feature = cols_C

# 全体におけるカウントを先に作っておく
for col in cols_feature:
    cnt_map = data[col].value_counts().to_dict()
    data[f"{col}_cnt"] = data[col].map(cnt_map)
    
    
def parallel_dtw(df, col, dtw):
    tmp = df[df[col_dtw]==dtw]

    # local count
    local_cnt_map = tmp[col].value_counts().to_dict()
    df[f"DT-W_{dtw}_{col}_cnt"] = df[col].map(local_cnt_map)
    df[f"DT-W_{dtw}_{col}_cnt_ratio_global"] = df[f"DT-W_{dtw}_{col}_cnt"] / df[f"{col}_cnt"]

    cols_save = [col for col in data.columns if col.count('_cnt') and col.count('DT-W')]
    train = data.iloc[:train_length][cols_save]
    test = data.iloc[train_length:][cols_save]
    save_feature(train[cols_save], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
    save_feature(test[cols_save],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)
    
    
for col in tqdm(cols_feature):
#     for dtw in list_dtw:
    Parallel(n_jobs)([delayed(parallel_dtw)(data[[col_dtw, col, f"{col}_cnt"]], col, dtw) for dtw in list_dtw])

0it [00:00, ?it/s]
